# Tratamento dos dados ANAC

    Optei por criar uma cell para o tratamento dos dados, sendo feita somente a troca dos diretórios dos arquivos eseus nomes. A escolha da forma de tratamento foi visando maior eficiência e na melhor forma da base chegar aos cientistas e analistas de dados com a ordenação coerente da base. Para garantir a segurança e integridade dos dados criei um novo csv para cada  base de dados tratado com a nomenclatura "Nome_Original_tratamento.csv".
    
        
 - Descrever qual estratégia você usaria para ingerir estes dados de forma incremental caso precise capturar esses dados a cada mes?
 
 Para automatizar o processo desenvolveria com a função os um laço de repetição na pasta com dados brutos e carregava os dados onde com um if  com os para outro diretório para verificar se no data lake ja tem um arquivo de mesmo nome, se houver não carrega, se não houver carrega e faz todo processo de crregar e enviar para o data lake.
 
 
    
## Desafios abordados
- Carregar dos dados de AIR_CIA
  - Normalizar o cabeçalho para snake case
  - Separar a coluna 'ICAO IATA' em duas colunas, seu conteúdo está separado por espaço e pode não conter o código IATA, caso não contenha o código IATA, deixe o valor nulo.
  - Salvar estes dados

In [2]:
# importe das bibliotecas
import pandas as pd
import numpy as np

## Tratamento ANAC

In [16]:
# Carregamento dos dados 
pd.set_option('display.max_columns',21)
df = pd.read_csv('C:/Users/Didrich/Documents/Python Scripts/DataEnginnerTest/AIR_CIA/ANAC_Brutos/ANAC_20211220_203643.csv', sep=';')

# Separação da coluna 'ICAO IATA' em duas colunas
df[['ICAO', 'IATA']] = df['ICAO IATA'].str.split(' ', n=1, expand=True)

# Substituir valores em branco na coluna 'IATA' por NaN
df['IATA'] = df['IATA'].apply(lambda x: None if x == '' else x)

# Remover a coluna 'ICAO IATA' original
df = df.drop(columns=['ICAO IATA'])

# Normalizar o cabeçalho para snake_case
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

colunas_ordenadas = [
    'razão_social',
    'icao',
    'iata',
    'cnpj',
    'atividades_aéreas',
    'endereço_sede',
    'telefone',
    'e-mail',
    'decisão_operacional',
    'data_decisão_operacional',
    'validade_operacional'
]
df = df[colunas_ordenadas]

# Exibir as primeiras linhas do DataFrame resultante
df.head()

,razão_social,icao,iata,cnpj,atividades_aéreas,endereço_sede,telefone,e-mail,decisão_operacional,data_decisão_operacional,validade_operacional
0,AEROSUL TÁXI AÉREO LTDA (EX.: AUSTEN TÁXI AÉRE...,ASO,2S,27.315.694/0001-02,"SERVIÇOS AÉREOS PÚBLICOS, TÁXI-AÉREO, TRANSPOR...","RODOVIA PR 218, KM 7, JARDIM UNIVERSITÁRIO, AE...",(43) 3176-4030,operacoes@aerosul.com.br,DECISÃO Nº 282,10/02/2021,NaN
1,ATA - AEROTÁXI ABAETÉ LTDA.,ABJ,None,14.674.451/0001-19,"TÁXI-AÉREO, TRANSPORTE AÉREO NÃO REGULAR, TRAN...","PÇA. GAGO COUTINHO, S/N, AEROPORTO INTERNACIO...",(71) 34629600 / (71) 996643563,guilherme@abaete.com.br,DECISÃO Nº 36,05/03/2020,NaN
2,OMNI TÁXI AÉREO S.A.,OMI,O1,03.670.763/0001-38,"AEROFOTOGRAFIA, AEROINSPEÇÃO, SERVIÇOS AÉREOS ...","AV. AYRTON SENNA, 2541 RUA F1 LOTE 40 - AEROPO...",(61) 98335-3444,lira@plfconsultoria.com.br,DECISÃO Nº 100,10/07/2019,12/07/2024
3,PRIME TÁXI AIR LTDA,NaN,NaN,23.568.370/0001-25,"TÁXI-AÉREO, TRANSPORTE AÉREO NÃO REGULAR, SERV...","AL. RIO NEGRO, 585 - BLOCO A - CJ.95 - SALA E,...",(11) 41958307 / (11) 41958308,mmatta@primefractionclub.com.br,DECISÃO Nº 237,17/12/2020,18/12/2025
4,SIDERAL LINHAS AÉREAS LTDA.,SID,0S,10.919.908/0001-57,"TRANSPORTE AÉREO NÃO REGULAR, TRANSPORTE AÉREO...","RODOVIA CONTORNO LESTE BR-116, Nº 9.119, 3º AN...",(41) 3116-9000 / (41) 3116-9059,office@siderallinhasaereas.com.br,DECISÃO Nº 37,05/03/2020,06/03/2025


In [17]:
# Especifique o caminho completo para a pasta onde deseja salvar o arquivo CSV
pasta_especifica = 'C:/Users/Didrich/Documents/Python Scripts/DataEnginnerTest/AIR_CIA/ANAC_tratado'

# Certifique-se de que a pasta existe, caso contrário, crie-a
if not os.path.exists(pasta_especifica):
    os.makedirs(pasta_especifica)

# Especifique o nome do arquivo CSV
nome_arquivo_csv = 'ANAC_20211220_203643_tratado.csv'

# Crie o caminho completo para o arquivo CSV
caminho_arquivo_csv = os.path.join(pasta_especifica, nome_arquivo_csv)

# Exporte o DataFrame para o arquivo CSV
df.to_csv(caminho_arquivo_csv, index=False, sep=";")  # index=False para não salvar o índice

print(f'O DataFrame foi exportado para {caminho_arquivo_csv}')


O DataFrame foi exportado para C:/Users/Didrich/Documents/Python Scripts/DataEnginnerTest/AIR_CIA/ANAC_tratado\ANAC_20211220_203643_tratado.csv


## Dados unificados ANAC
    Para analises futuras desenvolvi a união das tabelas.

In [23]:
# Carregamento dos dados 
dados1 = pd.read_csv('C:/Users/Didrich/Documents/Python Scripts/DataEnginnerTest/AIR_CIA/ANAC_tratado/ANAC_20211220_203627_tratado.csv', sep=';')
dados2 = pd.read_csv('C:/Users/Didrich/Documents/Python Scripts/DataEnginnerTest/AIR_CIA/ANAC_tratado/ANAC_20211220_203643_tratado.csv', sep=';')
dados3 = pd.read_csv('C:/Users/Didrich/Documents/Python Scripts/DataEnginnerTest/AIR_CIA/ANAC_tratado/ANAC_20211220_203733_tratado.csv', sep=';')

# União dos DataFrames
df_anac = pd.concat([dados1, dados2, dados3], ignore_index=True)


In [25]:
# Especifique o caminho completo para a pasta onde deseja salvar o arquivo CSV
pasta_especifica = 'C:/Users/Didrich/Documents/Python Scripts/DataEnginnerTest/AIR_CIA/ANAC_tratado'

# Certifique-se de que a pasta existe, caso contrário, crie-a
if not os.path.exists(pasta_especifica):
    os.makedirs(pasta_especifica)

# Especifique o nome do arquivo CSV
nome_arquivo_csv = 'df_anac.csv'

# Crie o caminho completo para o arquivo CSV
caminho_arquivo_csv = os.path.join(pasta_especifica, nome_arquivo_csv)

# Exporte o DataFrame para o arquivo CSV
df.to_csv(caminho_arquivo_csv, index=False, sep=";")  # index=False para não salvar o índice

print(f'O DataFrame foi exportado para {caminho_arquivo_csv}')


O DataFrame foi exportado para C:/Users/Didrich/Documents/Python Scripts/DataEnginnerTest/AIR_CIA/ANAC_tratado\df_anac.csv
